In [1]:
import torch
from transformers import BertTokenizer, BertModel

# Data preprocessing

## How to load tokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('./pytorch-ernie', do_lower_case = True)

### Convert batch sentence to tokens

In [3]:
batch_sentences = ["Hello I'm a single sentence",
                   "And another sentence",
                   "And the very very last one"]
encoded_inputs = tokenizer(batch_sentences, padding=True, return_tensors="pt", add_special_tokens=True)

In [4]:
encoded_inputs

{'input_ids': tensor([[    1,  6368,  1675, 17963,  1979,  1545,  7512,  8090,  9595, 10483,
             2],
        [    1,  1662,  7076,  8090,  9595, 10483,     2,     0,     0,     0,
             0],
        [    1,  1662,  1499,  6318,  6318,  6975,  3777,     2,     0,     0,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

## Modeling

In [8]:
from Code.model import *

In [9]:
model = BertClassifier()

In [16]:
batch_sentences = ["Hello I'm a single sentence",
                   "And another sentence",
                   "And the very very last one"]
encoded_inputs = tokenizer(batch_sentences, padding=True, return_tensors="pt", add_special_tokens=True)
for ids in encoded_inputs["input_ids"]:
    print(tokenizer.decode(ids))
input_tensor = encoded_inputs["input_ids"]
padding_mask = encoded_inputs['attention_mask']

[CLS] hello i [UNK] m a single sentence [SEP]
[CLS] and another sentence [SEP] [PAD] [PAD] [PAD] [PAD]
[CLS] and the very very last one [SEP] [PAD] [PAD] [PAD]


In [17]:
padding_mask
encoded_inputs["input_ids"]

tensor([[    1,  6368,  1675, 17963,  1979,  1545,  7512,  8090,  9595, 10483,
             2],
        [    1,  1662,  7076,  8090,  9595, 10483,     2,     0,     0,     0,
             0],
        [    1,  1662,  1499,  6318,  6318,  6975,  3777,     2,     0,     0,
             0]])

In [18]:
output = model(input_tensor)
print(output.detach().numpy().shape)

(3, 11)


In [19]:
y_true = output
mask_BCE_loss(y_true, output, padding_mask)

tensor(0.5048, grad_fn=<MulBackward0>)

## Data loader

In [206]:
import torch
from transformers import BertTokenizer, BertModel
import glob
import chinese_converter
from string import punctuation
import ast
import numpy as np

class AMLDataset(torch.utils.data.Dataset):
    def __init__(self, data_path='./data/training_set/', model_path='./pytorch-ernie'):
        self.paths = glob.glob(data_path+"*.txt")
        self.tokenizer = BertTokenizer.from_pretrained('./pytorch-ernie', do_lower_case = True)
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        with open(path, encoding = 'utf-8') as f:
            text = f.read().split('\n')
            target = ast.literal_eval(text[0])
            input_data = "".join(text[1:])
            # remove punctuation
            punctuation_characters = punctuation+'。、·「」！，）：（【】'
            input_data = input_data.translate(str.maketrans('', '',punctuation_characters ))
            # remove space
            input_data = input_data.replace(' ','')
        return input_data, target
        

def convert_batch_token_target(batch_input_data, batch_target):
    print(batch_target)
    batch_input_data = [chinese_converter.to_simplified(d) for d in batch_input_data]
    input_tensor = tokenizer(batch_input_data, padding=True, return_tensors="pt", add_special_tokens=True)
    return input_tensor["input_ids"], input_tensor['attention_mask'], batch_target
    

In [207]:
data = AMLDataset()
dataloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=1)

In [208]:
for d, t in dataloader:
    print(t, d)
    d, m, t = convert_batch_token_target(list(d), list(t))
    print(d.size(), m.size())
    break

[] ('75％凶槍是改造操作槍嚴重影響治安為達最佳瀏覽效果建議使用ChromeFirefox或MicrosoftEdge的瀏覽器〔記者邱俊福／台北報導〕319槍擊案之前歹徒多以市售模擬槍改造的槍枝犯案經修法管制模擬槍後部分廠商業者為規避法律規定製作形式與材質類似真槍的玩具槍商品與模擬槍差別僅在於移除撞針的打擊構造自稱為操作槍但事實上只要具備簡單車床技術打通槍管放上撞針便可化身為具有殺傷力改造槍械造成近年來改造槍械抓不勝抓原本經濟部於1992年發布玩具槍管理規則針對外觀近似真槍的玩具槍加強管理不過經大法官釋憲認為該規則為內規不能限制業者便在2002年廢止該規則自此玩具槍無法源規範開始流行跟真槍相似但槍管有阻鐵的模擬槍當時只要打通槍管再製作土造子彈便成為1支有殺傷力的改造槍械319槍擊案中打傷總統陳水扁的就是這種改造手槍不過319槍擊案發生後為管制模擬槍增訂槍砲彈藥刀械管制條例第20條之1規定指具打擊底火且外型構造材質類似真槍者為模擬槍對於該槍加以管制近年來部分廠商業者移除撞針即不具打擊底火變成自稱的操作槍而經警方鑑驗槍擊案凶槍發現有超過7成5的凶槍是歹徒以酷似真槍的操作槍改造而成嚴重影響社會治安', '', '崁頂鄉長林光華共11人涉貪遭起訴屏東地檢署偵辦崁頂鄉長林光華貪瀆案查出林涉嫌自2017年至2018年間親自或透過崁頂鄉代會主席郭茂良以120萬元至150萬元不等金額作為約定錄取鄉公所人員價碼共計收受400萬元賄款屏檢27日偵查終結全案依貪汙罪起訴11人屏檢偵辦此件貪瀆案總計執行搜索6次共計搜索49處所偵訊被告及相關證人共計161人次包括鄉長代表會主席全被收押禁見在地方引發議論除了議價職缺外檢調還發現林光華自2016年至2019年間辦理鄉內堤防工程採購案開標前洩漏相關資訊給投標協會分別圖利數十萬元同時在環境整修工程標案拿了包商39萬元另在殯儀館屋頂修繕工程服務中心屋頂防水暨周邊環境保固改善工程採購活動中心圍牆及大門增設工程採購案及國小運動場整修工程採購等案件檢調皆查出有黑手伸入郭茂良擔心投標未滿3家而流標還要求其他廠商陪標屏檢陸續清查相關涉案人員到案釐清案情經檢察官偵查終結認被告林光華郭茂良鄉公所祕書林全明及陳文君等11人涉嫌貪汙等罪提起公訴消息傳回地方有政壇人事指出崁頂鄉長從第11任王龍興至16任林家和共6任都只做一任不是放棄競選不然就連任失敗直到第17任林光

In [97]:
pos = 0
neg = 0 
for d in data:
    ids, mask, target = d
    if len(target) > 0:
        pos += 1
    else:
        neg += 1
print(pos, neg, pos+neg)
    
    

372 4651 5023


In [68]:
with open("./data/training_set/1.txt", encoding = 'utf-8') as f:
    text = f.read().split('\n')
    target = text[0]
    input_data = chinese_converter.to_simplified("".join(text[1:]))
    # remove punctuation
    punctuation_characters = punctuation+'。、·「」！，）：（'
    input_data = input_data.translate(str.maketrans('', '',punctuation_characters ))
    # remove space
    input_data = input_data.replace(' ','')
    # convert to batch
    input_data = [input_data]
    input_tensor = tokenizer(input_data, padding=True, return_tensors="pt", add_special_tokens=True)
    print(input_data)

['量化交易追求绝对报酬有效对抗牛熊市近年来投资市场波动越来越明显追求低波动绝对报酬的量化交易备受注目专家表示采用量化交易策略投资台股不管是处于多头或是空头市场绩效及波动度均可领跑大盘甚至比国内投资台股的股票型基金及ETF的波动率还低表现也更为稳定大数据时代来临风行欧美50年的量化交易俨然成为显学台湾亦开始重视此一趋势发展也因此中华机率统计学会及台北科技大学管理学院携手主办并由元大期货摩根亚太量化交易等公司担任协办单位今75日举办时间序列与量化交易研讨会就目前热门的量化交易智能投资等相关议题进行研讨越来越多的基金公司重视量化交易全球规模较大的避险基金多采行量化交易包括桥水基金BridgewaterAssociatesAQR资产管理公司曼氏集团ManGroup文艺复兴科技RenaissanceTechnologies等全球知名避险基金摩根亚太集团董事长张尧勇指出避险基金规模约为5兆美元采取量化交易的基金规模约1兆美元比重占了20代表量化交易的操作绩效好才会有那么高的比重量化交易的操作绩效不亚于价值投资及技术投资被誉为数学天才最赚钱的基金经理人文艺复兴对冲基金创始人詹姆斯西蒙斯JamesSimons所管理的大奖章Medallion基金便是典型的量化交易绩效表现优异不仅胜过索罗斯的量子基金也打败了股神巴菲特的价值投资近年来台湾也逐渐重视量化交易摩根亚太量化交易公司今年开始将量化交易引进台股投资是国内首家推出量化交易策略的公司初期对象锁定法人机构进行私募投资张尧勇表示数学不只是一门学科更是一项扭转干坤转败为胜的竞争利器量化交易就是将数学运用在股市投资透过复杂精密的推理计算打败股市他指出目前全球利率水平50位于年来新低美国十年期公债殖利率只有2日本及一些欧洲国家甚至是负利率历史极低利率带导致无风险及低风险工具的投资报酬率太低例如银行存款政府公债及投资等级公司债无法对抗通货膨胀及支付负债所以资产配置必须增加较高风险的投资如股票高收益公司债或新兴市场债券但是这些投资工具波动很大而且很容易产生亏损他表示长期来说股市绝对是好的投资报酬率也不差但是如何选股是一门学问目前全球60个主要股市交易所总市值70兆美元而台湾上市上柜市值35兆台币面对全球及台湾最大的股市金矿投资人应该善用量化交易这个超级挖矿机以程式选股来掏金张尧勇指出量化投资具有低波动绝对报酬优于大盘的三大特点摩根亚太量化交易模

In [69]:
print(len(input_data[0]))
print(len(input_tensor["input_ids"].numpy()[0]))

1438
1361


In [70]:
for ids in input_tensor["input_ids"]:
    decode_text = tokenizer.decode(ids)

In [74]:
decode_text.split(' ')

['[CLS]',
 '量',
 '化',
 '交',
 '易',
 '追',
 '求',
 '绝',
 '对',
 '报',
 '酬',
 '有',
 '效',
 '对',
 '抗',
 '牛',
 '熊',
 '市',
 '近',
 '年',
 '来',
 '投',
 '资',
 '市',
 '场',
 '波',
 '动',
 '越',
 '来',
 '越',
 '明',
 '显',
 '追',
 '求',
 '低',
 '波',
 '动',
 '绝',
 '对',
 '报',
 '酬',
 '的',
 '量',
 '化',
 '交',
 '易',
 '备',
 '受',
 '注',
 '目',
 '专',
 '家',
 '表',
 '示',
 '采',
 '用',
 '量',
 '化',
 '交',
 '易',
 '策',
 '略',
 '投',
 '资',
 '台',
 '股',
 '不',
 '管',
 '是',
 '处',
 '于',
 '多',
 '头',
 '或',
 '是',
 '空',
 '头',
 '市',
 '场',
 '绩',
 '效',
 '及',
 '波',
 '动',
 '度',
 '均',
 '可',
 '领',
 '跑',
 '大',
 '盘',
 '甚',
 '至',
 '比',
 '国',
 '内',
 '投',
 '资',
 '台',
 '股',
 '的',
 '股',
 '票',
 '型',
 '基',
 '金',
 '及',
 'etf',
 '的',
 '波',
 '动',
 '率',
 '还',
 '低',
 '表',
 '现',
 '也',
 '更',
 '为',
 '稳',
 '定',
 '大',
 '数',
 '据',
 '时',
 '代',
 '来',
 '临',
 '风',
 '行',
 '欧',
 '美',
 '50',
 '年',
 '的',
 '量',
 '化',
 '交',
 '易',
 '俨',
 '然',
 '成',
 '为',
 '显',
 '学',
 '台',
 '湾',
 '亦',
 '开',
 '始',
 '重',
 '视',
 '此',
 '一',
 '趋',
 '势',
 '发',
 '展',
 '也',
 '因',
 '此',
 '中',
 '华',
 '机',
 '率',
 '统